In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('data_training_full.csv', header=0, index_col=0)
df.head()

In [5]:
def convert_to_format(row):
    sentence1 = row['text']
    sentence2 = row['category']
    prompt = """Below is an instruction that describes a task paired with input that provides further context. Write a response that appropriately completes the request."""
    instruction = """Given the following sentence, your job is to label it with one or multiple underlying human values according to Schwartz Value Theory. They human values are implict present in the argument. The answer should have the following format: Values: [value1, value2, ... ]. All available human values are: "Self-direction: thought, Self-direction: action, Stimulation, Hedonism, Achievement, Power: dominance, Power: resources, Face, Security: personal, Security: societal, Tradition, Conformity: rules, Conformity: interpersonal, Humility, Benevolence: caring, Benevolence: dependability, Universalism: concern, Universalism: nature, Universalism: tolerance, Universalism: objectivity" . Use only these labels in your answer."""
    input = str(sentence1)
    response = f"""```json
        {{'argument': '{sentence1}', 'human_values': '{sentence2}'}}
        ```
        """
    if len(input.strip()) == 0:  #  prompt + 2 new lines + ###instruction + new line + input + new line + ###response
        text = prompt + "\n\n### Instruction:\n" + instruction + "\n### Response:\n" + response
    else:
        text = prompt + "\n\n### Instruction:\n" + instruction + "\n### Input:\n" + input + "\n" + "\n### Response:\n" + response
    
    # we need 4 columns for auto train, instruction, input, output, text
    return pd.Series([instruction, input, response, text])

In [6]:
new_df = df.apply(convert_to_format, axis=1)
new_df.columns = ['instruction', 'input', 'output', 'text']

In [8]:
new_df.to_csv("data_instruct_format.csv", index=False)

In [12]:
print(new_df["text"][0])

Below is an instruction that describes a task paired with input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Given the following sentence, your job is to label it with one or multiple underlying human values according to Schwartz Value Theory. They human values are implict present in the argument. The answer should have the following format: Values: [value1, value2, ... ]. All available human values are: "Self-direction: thought, Self-direction: action, Stimulation, Hedonism, Achievement, Power: dominance, Power: resources, Face, Security: personal, Security: societal, Tradition, Conformity: rules, Conformity: interpersonal, Humility, Benevolence: caring, Benevolence: dependability, Universalism: concern, Universalism: nature, Universalism: tolerance, Universalism: objectivity" . Use only these labels in your answer.
### Input:
we should ban human cloning as it will only cause huge issues when you have a bunch of the same h

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [14]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [15]:
from google.colab import drive

drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/data_instruct_format.csv')

ModuleNotFoundError: No module named 'google.colab'

In [16]:
from datasets import Dataset

dataset = Dataset.from_pandas(new_df)

In [17]:
dataset

Dataset({
    features: ['instruction', 'input', 'output', 'text', '__index_level_0__'],
    num_rows: 6876
})

In [19]:
dataset = dataset.remove_columns("__index_level_0__")

In [36]:
# The model that you want to train from the Hugging Face hub
model_name = "meta-llama/Llama-2-7b-hf"
# model_name = "abhishek/llama-2-7b-hf-small-shards"

# Fine-tuned model name
new_model = "llama-2-qlora-value-detector"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 1e-5

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 100

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [29]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
print(compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

torch.float16


In [30]:
from huggingface_hub import login
login()

In [ ]:
from load_tokenizer import load_tokenizer
tokenizer = load_tokenizer(model_name, trust_remote_code=True, use_slow=True)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

In [ ]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",  # this is the text column in dataset 
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

In [ ]:
# Train model
trainer.train()

In [ ]:
trainer.model.save_pretrained(new_model)


In [ ]:
from peft import get_peft_model
lora_config = LoraConfig.from_pretrained(new_model)
model = get_peft_model(model, lora_config)
model.push_to_hub(new_model)

In [ ]:
!zip -r llama-contradictor.zip results llama-contradictor
!mv llama-contradictor.zip /content/drive/MyDrive

In [ ]:
!unzip /content/drive/MyDrive/llama-contradictor.zip -d .

## Reload Model

In [ ]:
import torch
from peft import PeftModel    
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer

In [ ]:
model_name = "meta-llama/Llama-2-7b-hf"
adapters_name = "danschr/llama-2-qlora-value-detector"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)

model = PeftModel.from_pretrained(base_model, '/content/llama-contradictor')
model = model.merge_and_unload()
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

In [ ]:
prompt_template = """### Instruction:
Given the following sentence, your job is to label it with one or multiple underlying human values according to Schwartz Value Theory. They human values are implict present in the argument. The answer should have the following format: Values: [value1, value2, ... ]. All available human values are: "Self-direction: thought, Self-direction: action, Stimulation, Hedonism, Achievement, Power: dominance, Power: resources, Face, Security: personal, Security: societal, Tradition, Conformity: rules, Conformity: interpersonal, Humility, Benevolence: caring, Benevolence: dependability, Universalism: concern, Universalism: nature, Universalism: tolerance, Universalism: objectivity" . Use only these labels in your answer.
### Input:
{}

### Response:
"""

sentence = dataset[10]["input"]
input_sentence = prompt_template.format(sentence.strip())
print(input_sentence)

In [ ]:
device = "cuda:0"

inputs = tokenizer(input_sentence, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
pipe = pipeline(task="text-generation", model=m, tokenizer=tokenizer, max_length=300)
results = pipe(input_sentence)
print(results)

In [ ]:
import re
import json

def format_results(s):
  pattern = r'```json\n(.*?)\n```'

  # Find all occurrences of JSON objects in the string
  json_matches = re.findall(pattern, s, re.DOTALL)
  if not json_matches:
    # try to find 2nd pattern
    pattern = r'\{.*?"sentence":.*?"negation":.*?\}'
    json_matches = re.findall(pattern, s)

  # Return the first JSON object found, or None if no match is found
  return json.loads(json_matches[0]) if json_matches else None